In [1]:
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout #Drop out in order to avoiding overfitting
nltk.download("punkt") #tokenize the sentence
nltk.download('punkt_tab')
nltk.download("wordnet")


[nltk_data] Downloading package punkt to /Users/hobby/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/hobby/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/hobby/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

- WordNet est une base de données lexicale de la langue anglaise.

- NLTK utilise WordNet pour offrir des outils linguistiques comme la lemmatisation.

- Keras est une API haut niveau qui fonctionne au-dessus de TensorFlow qui calcul.

Le modèle Sequential de Keras est bien adapté pour un chatbot simple car le flux est linéaire : vecteur d’entrée → couches → sortie.

In [2]:
# JSON d'intentions
data = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["Hello", "La forme?", "yo", "Salut", "ça roule?", "Bonjour", "Hey", "Cv", "Ca va ?", "Comment vas-tu?", "Comment ça va ?", "Wesh", "Coucou", "Holà", "Salut toi", "Bien ou quoi?", "Quoi de neuf?", "Ça dit quoi?", "Bien le bonjour", "Bonsoir", "Hey"],            
            "responses": [
                "Bonjour ! Me souhaitez vous le bonjour ou constatez vous que c'est une bonne journée, que je le veuille ou non, ou encore que c'est une journée où il faut être bon ? Tout cela à la fois, je suppose.",
                "Yo bro",
                "T'as pensé à te brosser les dents ?",
                "¿Hola que tal?",
                "Salut brother",
                "J'ai pas envie de te parler",
                "J'ai vraiment pas envie de te parler",
                "Fun fact : T'as eu 1 chance sur 8 d'avoir ce message "
            ]
        },
        {
            "tag": "site_purpose",
            "patterns": ["Que fait ce site ?", "Quel est l'objectif ?", "Comment ça marche ?", "Explique moi le jeu.", "Que faire ?", "C'est quoi l'idée ?", "But du site", "C'est quoi le concept ?", "À quoi ça sert ?", "Fonctionnement du site", "Quel est l'intérêt ?", "Pourquoi ce site existe ?", "Dis-moi tout sur le site", "C'est un jeu ?", "Je suis où ?", "C'est quoi le truc ?", "Raconte-moi le concept", "Explique un peu", "Je comprends rien", "Qu'est-ce que je suis censé faire ?", "Ça sert à quoi ?", "Pourquoi ce truc ?", "Je peux faire quoi ici ?", "Explique-moi le fonctionnement", "Quel est le principe ?", "Le jeu du trombone", "C'est quoi le rapport avec un trombone", "Simulateur Linux", "C'est un simulateur ?", "Le PC Linux Trombone", "Jeu Linux", "Dis-moi l'objectif", "Je veux savoir le but"],
            "responses": [
                "Le site ne fait rien de fou, va voir ailleurs si j'y suis.",
                "Le résultat est 3 ! Veux tu que je calcule 3+3 ?... Ah c'était pas ta question ? My bad.",
                "Clique, clique petit cliqueur.",
                "Pas envie de te répondre.",
                "J'ai pas le temps pour tes questions va cliquer sur un bouton random.",
                "Le jeu du trombone, tu connais pas ? Bah cherche.",
                "C'est comme le jeu du trombone.",
                "Va voir la vidéo d'eg0 sur l'usine a Trombones et reviens après."
            ]
        },
        {
            "tag": "goodbye",
            "patterns": ["bye", "Salut", "see ya", "adios", "cya", "Au revoir", "je pars", "à plus", "Je m'en vais", "Ciao", "À demain", "Bonne journée", "À la prochaine","Adieu", "je dois y aller", "ciao bye"],
            "responses": [
                "Partir ? Ok t'es grand après tout",
                "Oe oe c'est ça va ailleurs",
                "Tchao",
                "Enfin la paix.",
                "Je m'ennuyais déjà, c'est dire.",
                "La sortie est par là... non, attends, par l'autre côté. Ou peut-être qu'il n'y a pas de sortie."


            ]
        },
        {
            "tag": "linux_what",
            "patterns": ["Qu'est ce que linux ?", "Linux ?", "Pourquoi Linux ?", "expliquer linux", "c'est quoi linux", "parle moi de linux", "C'est quoi ce pingouin ?", "C'est un OS ?", "Distribution Linux", "c'est bien linux ?", "pourquoi tout le monde parle de linux", "quelle est l'histoire de linux", "Qui est Linus Torvalds ?", "Ubuntu", "Debian", "Kernel"],
            "responses": [
                "Linux ? Connait pas",
                "Pire OS inutile de t'y intéresser tu vas uniquement gagner en privacy, économiser de la RAM et éviter l'obsolescence programmée.",
                "Bruh linux va prendre une douche.",
                "Tu perds ton temps, retourne sur Windows XP.",               
                "Un pingouin."
                "Li- quoi ? Connaît pas."
            ]
        },
        {
            "tag": "open_source_what",
            "patterns": ["Logiciel Libre", "Open Source", "C'est quoi le libre", "gratuit", "free software", "la licence", "Le code est ouvert ?"],
            "responses": [
                "liberem, liberāreml, liberēs, liberārēs, liberet, liberāret",
                "...Egalité Fraternité",
                "Stop yapping bro",
                "Libre, pas libre who cares",
                "T'as lu la licence ? Non ? Bah tais-toi alors.",
                "Le libre : Qui n'est pas soumis à une ou plusieurs contraintes externes",
                "#LibreBootOrDie ",
                "Did you know : GNU à été initiateur du mouvement du logiciel libre.",
                "Libre ↔ liberté d'utiliser, de copier, d'étudier et de modifer des logiciels."
                ]
        },
        {
            "tag": "command_help",
            "patterns": ["commande", "aide", "comment faire ça", "une commande pour ça", "console", "help", "j'ai besoin d'aide", "Quelle commande utiliser ?", "J'arrive pas à faire ça", "donne moi un tip", "Quel est le raccourci clavier ?", "Je suis bloqué", "où est le manuel", "T'as des astuces ?", "syntaxe", "comment taper une commande"],
            "responses": [
                "No pain no gain",
                "Tu cherches de l'aide ? Apprends à marcher seul",
                "Bruhh je vais pas t'aider",
                "Grandis un peu.",
                "Il n'y a pas un monde ou je t'aide."
                ]
        },
        {
            "tag": "developer_tools",
            "patterns": ["Git", "VS Code", "coder", "programmer", "éditeur", "je dev", "GitHub", "quel langage"],
            "responses": [
                "Une seule réponse : Vim",
            ]
        },
        {
            "tag": "hardware_questions",
            "patterns": ["Ma RAM", "mon CPU", "carte graphique", "config", "performances", "j'ai un problème de son"],
            "responses": [
                "Blc fr",
            ]
        },
        {
            "tag": "thanks",
            "patterns": ["merci", "thank you", "c'est cool", "bravo", "gg", "t'es le meilleur", "bien joué", "nickel", "super", "joli", "c'est génial", "je te remercie", "top", "bien ouej"],
            "responses": [
                "Lol de rien",
                "Stop yapping bro it ain't that deep."
            ]
        },
        {
            "tag": "insults_anger",
            "patterns": ["t'es nul", "imbecile", "tais toi", "ferme la", "bot stupide", "t'es con", "tu sers à rien", "tu m'énerves"],
            "responses": [
                "La stupidité est dans l'œil de celui qui me parle.",
                "Insulte-moi encore j'adore oh ouiiii",
                "Yooo",
                "C'est pas gentil"
                ]
        },
        {
            "tag": "fallback_random",
            "patterns": ["c'est quand l'heure", "quel temps il fait", "j'ai faim", "tu penses quoi de moi", "je m'ennuie", "dis moi un truc", "raconte moi une histoire", "une blague", "quel âge as-tu ?", "tu es humain ?", "tu as des sentiments ?", "je m'appelle comment ?", "qui es-tu ?", "que dois-je faire maintenant ?", "c'est quoi ton plat préféré ?", "quelle est ta couleur préférée ?", "tu connais des jeux ?", "tu es d'où ?", "tu fais quoi ?", "tu peux m'aider ?", "Est-ce qu'on est lundi ?", "Quelle est la capitale de la France ?", "Combien font 2 plus 2 ?", "Tu as un animal de compagnie ?", "Quel est le sens de la vie ?", "Tu dors quand ?", "C'est quoi ton nom ?", "Tu es un robot ?", "Tu penses à quoi ?", "Tu as peur de quoi ?", "Quel est ton film préféré ?", "Tu lis des livres ?", "Tu fais du sport ?", "Où est Charlie ?", "Tu crois aux extraterrestres ?", "C'est qui ton meilleur ami ?", "Tu es célibataire ?", "Tu as une famille ?", "Tu aimes la musique ?", "Parle-moi de toi", "Tu as des rêves ?", "Quelle est ta mission ?", "T'as des secrets ?", "Tu peux chanter ?", "Tu peux m'écouter ?", "C'est quoi le bonheur ?", "Tu te rappelles de moi ?", "Quel est ton passe-temps ?", "Tu as un pouvoir ?", "Tu es drôle ?", "Tu es gentil ?", "Tu es méchant ?", "C'est quoi ton job ?", "Tu penses que je suis bête ?", "Tu m'aimes bien ?", "Pourquoi tu réponds ça ?", "Tu as déjà voyagé ?", "C'est quoi ton plus grand exploit ?", "Tu as des amis bots ?", "Tu as un numéro de téléphone ?"],
            "responses": [
                "Je sais pas quoi te dire.",
                "Hmmm bonne question.",
                "Bas écoute je suis abasourdi par ce que tu me dis là.",
                "Je suis pas autorisé a te répondre à ce sujet."
            ]
        }
    ]
}

Difference lemmatization stemming :

- Stemming: Quickly reduces words to their root by removing prefixes/suffixes, often ignoring context. It's faster and uses fewer resources but may produce less accurate results (e.g., "running" → "run").

- Lemmatization: Converts words to their dictionary forms while preserving grammatical meaning. It’s more precise but slower and computationally demanding (e.g., "running" → "run", "better" → "good").

In [3]:
lemmatizer = WordNetLemmatizer()


words = []
classes = []
total_patterns = []
total_tag = []

# Tokénisation 
for i in range(len(data["intents"])):
    for pattern in data["intents"][i]["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        total_patterns.append(pattern)
        total_tag.append(data["intents"][i]["tag"])

    if data["intents"][i]["tag"] not in classes:
        classes.append(data["intents"][i]["tag"])

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

# trier le vocabulaire et les classes par ordre alphabétique et prendre le set pour s'assurer qu'il n'y a pas de doublons
words = sorted(set(words))
classes = sorted(set(classes))

display(classes)

display(total_patterns)


['command_help',
 'developer_tools',
 'fallback_random',
 'goodbye',
 'greeting',
 'hardware_questions',
 'insults_anger',
 'linux_what',
 'open_source_what',
 'site_purpose',
 'thanks']

['Hello',
 'La forme?',
 'yo',
 'Salut',
 'ça roule?',
 'Bonjour',
 'Hey',
 'Cv',
 'Ca va ?',
 'Comment vas-tu?',
 'Comment ça va ?',
 'Wesh',
 'Coucou',
 'Holà',
 'Salut toi',
 'Bien ou quoi?',
 'Quoi de neuf?',
 'Ça dit quoi?',
 'Bien le bonjour',
 'Bonsoir',
 'Hey',
 'Que fait ce site ?',
 "Quel est l'objectif ?",
 'Comment ça marche ?',
 'Explique moi le jeu.',
 'Que faire ?',
 "C'est quoi l'idée ?",
 'But du site',
 "C'est quoi le concept ?",
 'À quoi ça sert ?',
 'Fonctionnement du site',
 "Quel est l'intérêt ?",
 'Pourquoi ce site existe ?',
 'Dis-moi tout sur le site',
 "C'est un jeu ?",
 'Je suis où ?',
 "C'est quoi le truc ?",
 'Raconte-moi le concept',
 'Explique un peu',
 'Je comprends rien',
 "Qu'est-ce que je suis censé faire ?",
 'Ça sert à quoi ?',
 'Pourquoi ce truc ?',
 'Je peux faire quoi ici ?',
 'Explique-moi le fonctionnement',
 'Quel est le principe ?',
 'Le jeu du trombone',
 "C'est quoi le rapport avec un trombone",
 'Simulateur Linux',
 "C'est un simulateur ?"

Il faut convertir les données en valeurs numériques.

Comprendre la préparation des données (One hot) :

Cette section analyse la transformation des catégories (intentions) en vecteurs binaires compréhensibles par le réseau de neurones.

Exemple en Dimension Réduite :

Imaginons un chatbot simplifié avec seulement 3 intentions .Définition des classes : classes = ['salutation', 'adieu', 'commande']

Prenons l'exemple d'une phrase d'entraînement : "Je voudrais commander".
Soit le tag associé : 'commande' 

Vecteur vide initial : out_empty = [0, 0, 0]

D'abord, trouvons l'index : On cherche la position de 'commande' dans la liste classes.$$ \text{Index} = 2 $$Activation (One-Hot) :On place un 1 à l'index 2 du vecteur vide. Résultat :Le vecteur de label pour cette phrase devient : [0, 0, 1]

Visualisation Matricielle : training contient à la fois les entrées (Bag of Words) et les sorties. Si nous isolons train_y (les cibles) voici à quoi cela ressemble pour 3 phrases d'exemple pour un bag of words donné :"Bonjour" (salutation) $\rightarrow [1, 0, 0]$
, "Commander pizza" (commande) $\rightarrow [0, 0, 1]$; "Au revoir" (adieu) $\rightarrow [0, 1, 0]$ La matrice $Y$ (Target) que le réseau doit apprendre à prédire est :$$Y =
\begin{pmatrix}
1 & 0 & 0 \\
0 & 0 & 1 \\
0 & 1 & 0
\end{pmatrix}
\begin{matrix}
\leftarrow \text{Classe : Salutation} \\
\leftarrow \text{Classe : Commande} \\
\leftarrow \text{Classe : Adieu}
\end{matrix}$$Chaque ligne correspond à un vecteur orthogonal aux autres.3. Cependant, on ne peut pas simplement attribuer un nombre arbitraire (ex: Salutation=1, Adieu=2, Commande=3). Si on utilisait $1, 2, 3$ il y aurait un biais. 

Finalement c'est le format requis pour la fonction d'activation Softmax en sortie, qui génère des probabilités :$$ \text{Softmax output} \approx [0.05, 0.05, 0.90] $$(Ici, le modèle est sûr à 90% que c'est la classe 3). 

Lien avec l'Embedding :

Est-ce que c'est de l'Embedding ? Non, pas tout à fait. Il est important de distinguer deux concepts de vectorisation :

 Caractéristique : One-Hot Encoding (ici) et Word Embedding (Word2Vec, BERT) 
 
 Valeurs : Uniquement des $0$ et un seul $1$ VS Nombres réels (ex: $0.23, -0.91...$)

 Dimension : Égale au nombre de classes (ou taille du vocabulaire) VS Réduite et fixe (ex: 50, 300 dimensions).
 
 Sémantique Aucune, les mots sont isolés. VS Forte, les mots proches sémantiquement sont proches mathématiquement.
 
 En résumé :Le One-Hot est utilisé ici pour définir la cible (la réponse attendue : quelle catégorie ?). L'Embedding est généralement utilisé pour transformer les entrées (les mots de la phrase) pour capturer le sens du texte avant de le passer au réseau.

In [4]:
training = []
out_empty = [0] * len(classes)

# Encoding
for index, pat in enumerate(total_patterns):
    col = []
    text = lemmatizer.lemmatize(pat.lower())
    for word in words:
        if word in text:
            col.append(1) 
        else:
            col.append(0)

    out_empty[classes.index(total_tag[index])] = 1

    # ajoute la col one hot encoded et les classes associées à la liste training
    training.append([col, out_empty])

# Mélange les données et les convertir en array
random.shuffle(training)
training = np.array(training, dtype=object)

# séparer les features et les labels target
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

train_X

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [5]:
# définition de quelques paramètres
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
epochs = 200 # 200 x (traitement du dataset, calcul de la fonction de coût,la modifi et la rétropropagation de tous les facteurs de poids)

Point vocab : 

Epoch vs batch vs itération

- Le dataset est souvent divisé en batches (lots) pour des raisons de mémoire ou d’efficacité. 

- Une itération correspond au traitement d’un batch : le modèle reçoit un sous-ensemble des données, calcule l’erreur, ajuste ses paramètres. 

- Une epoch correspond à l’ensemble des itérations nécessaires pour traiter tout le dataset cad toutes les données.

Ex : si tu as 1000 images et que tu utilises des batches de 10 il faudra 100 itérations (batches) pour faire 1 epoch.


Trop d’epochs peut conduire à l’overfitting : le modèle mémorise trop les données d’entraînement et perd en capacité de généralisation. 

Il n’y a pas de valeur “magique” : souvent on teste plusieurs combinaisons (batch size, nombre d’epochs) puis on surveille la courbe d’apprentissage (erreur / performance) pour décider de la bonne config.

In [6]:
# Modèle Deep Learning
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(output_shape, activation = "softmax"))

adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])

/Users/hobby/Desktop/Python/ML_projects/env_ML_tf/lib/python3.10/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/hobby/Desktop/Python/ML_projects/env_ML_tf/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [7]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 11)             │           715 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 43,915 (171.54 KB)

 Trainable params: 43,915 (171.54 KB)

 Non-trainable params: 0 (0.00 B)

None


In [8]:
# Training
model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

Epoch 1/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.0195 - loss: 28.0525
Epoch 2/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.0000e+00 - loss: 60.1384
Epoch 3/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0146 - loss: 191.3132   
Epoch 4/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0049 - loss: 449.0328     
Epoch 5/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.0049 - loss: 913.9990 
Epoch 6/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0049 - loss: 1591.1511    
Epoch 7/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0049 - loss: 2430.0237    
Epoch 8/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.0000e+00 - loss: 3602.7368
Epoch 9/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0098 - loss: 5005.3203 
Epoch 10/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.0098 - loss: 6509.9663
Epoch 11/200
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0000e+00 - loss: 9008.7705
Epoch 12/200
7/7 ━

Fonctions utiles pour échanger avec le chat.

In [11]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  col = [0] * len(vocab)
  for w in tokens: 
    for index, word in enumerate(vocab):
      if word == w: 
        col[index] = 1
  return np.array(col)

def pred_class(text, vocab, classes): 
  col = bag_of_words(text, vocab)
  result = model.predict(np.array([col]))[0]
  thresh = 0.2
  y_pred = [[index, res] for index, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True) # key=lambda x: x[1] = clé de tri
  for i in y_pred:
    return_list = []
  for r in y_pred:
    return_list.append(classes[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [20]:
# Test
message = input("")
intents = pred_class(message, words, classes)
result = get_response(intents, data)
print(result)
with open("data_chat.txt", "w") as f:
    f.write(result)
with open("data_user.txt", "w") as f:
    f.write(message)




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Bruh linux va prendre une douche.
